In [1]:
import pandas as pd
import numpy as np
import csv
import zipfile

from matplotlib import pyplot as plt
%matplotlib inline
filepath = "../resources/food.csv.zip"
zf = zipfile.ZipFile(filepath)
food = pd.read_csv(zf.open('food.csv'), parse_dates=True, dtype=object, delimiter="\t", quoting=csv.QUOTE_NONE, encoding='utf-8')
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows',10)
food

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,packaging,packaging_tags,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,no_nutriments,additives_n,additives,additives_tags,additives_en,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,ingredients_that_may_be_from_palm_oil_tags,nutrition_grade_uk,nutrition_grade_fr,pnns_groups_1,pnns_groups_2,states,states_tags,states_en,main_category,main_category_en,image_url,image_small_url,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,-butyric-acid_100g,-caproic-acid_100g,-caprylic-acid_100g,-capric-acid_100g,-lauric-acid_100g,-myristic-acid_100g,-palmitic-acid_100g,-stearic-acid_100g,-arachidic-acid_100g,-behenic-acid_100g,-lignoceric-acid_100g,-cerotic-acid_100g,-montanic-acid_100g,-melissic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,-alpha-linolenic-acid_100g,-eicosapentaenoic-acid_100g,-docosahexaenoic-acid_100g,omega-6-fat_100g,-linoleic-acid_100g,-arachidonic-acid_100g,-gamma-linolenic-acid_100g,-dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,-oleic-acid_100g,-elaidic-acid_100g,-gondoic-acid_100g,-mead-acid_100g,-erucic-acid_100g,-nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,-sucrose_100g,-glucose_100g,-fructose_100g,-lactose_100g,-maltose_100g,-maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g
0,0000000003087,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,1474103866,2016-09-17T09:17:46Z,1474103893,2016-09-17T09:18:13Z,Farine de blé noir,NaN,1kg,NaN,NaN,Ferme t'y R'nao,ferme-t-y-r-nao,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:FR,en:france,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"en:to-be-completed, en:nutrition-facts-to-be-c...","en:to-be-completed,en:nutrition-facts-to-be-co...","To be completed,Nutrition facts to be complete...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000024600,http://world-en.openfoodfacts.org/product/0000...,date-limite-app,1434530704,2015-06-17T08:45:04Z,1434535914,2015-06-17T10:11:54Z,Filet de bœuf,NaN,2.46 kg,plastique,plastique,NaN,NaN,Filet de bœuf,fr:filet-de-boeuf,fr:Filet-de-boeuf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,France,en:france,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,unknown,"en:to-be-completed, en:

In [2]:
def no_null_objects(data, columns=None):
    if columns is None:
        columns = data.columns
    return data[np.logical_not(np.any(data[columns].isnull().values, axis=1))]

In [3]:
def splitDataFrameList(df, target_column, separator):
    def splitListToRows(row, row_accumulator, target_column, separator):
        split_row = row[target_column].split(separator)
        for s in split_row:
            new_row = row.to_dict()
            new_row[target_column] = s
            row_accumulator.append(new_row)
    new_rows = []
    df.apply(splitListToRows,axis=1,args = (new_rows,target_column,separator))
    new_df = pd.DataFrame(new_rows)
    return new_df

In [4]:
food_countries = splitDataFrameList(no_null_objects(food, ["countries_en", 'additives_n', 'allergens']), "countries_en", ",")
countries = food_countries["countries_en"]
countries_temp = (countries.unique())
countries_temp

array(['France', 'United Kingdom', 'Spain', 'United States', 'Belgium',
       'Netherlands', 'Ireland', 'Canada', 'Australia', 'Czech Republic',
       'Indonesia', 'Denmark', 'Norway', 'Sweden', 'Germany', 'Lebanon',
       'Finland', 'Portugal', 'Italy', 'Switzerland', 'fr:Angleterre',
       'French Polynesia', 'Romania', 'de:Scotland', 'European Union',
       'Slovenia', 'Allemagne', 'Poland', 'Peru', 'Morocco', 'Luxembourg',
       'Guadeloupe', 'French Guiana', 'Gabon', 'Tunisia', 'Martinique',
       'Réunion', 'New Caledonia', 'Saint Pierre and Miquelon', 'Greece',
       'Austria', 'Hungary', 'Russia', 'India', 'Armenia', 'China',
       'Saudi Arabia', 'Japan', 'Cyprus', 'Scotland', 'Estonia', 'Malta',
       'New Zealand', 'South Africa', 'United Arab Emirates', 'Mexico',
       'Serbia', 'Bulgaria', 'fr:Marseille-6', 'Brazil', 'Montenegro',
       'Algeria', 'Andorra', 'Iraq', 'Singapore', 'Thailand'], dtype=object)

In [5]:
# outliers 
#food_fat_outlier =food[food['fat_100g'].astype(float) + food['protiens_100g'].astype(float) + food['fat_100g'].astype(float) > 100]
#food_fat_outlier

food_carbohydrates_outlier =food[food['carbohydrates_100g'].astype(float) > 100]
food_carbohydrates_outlier

food_fat_outlier =food[food['fat_100g'].astype(float) > 100]
food_fat_outlier

#food_sugars_outlier =food[food['proteins_100g'].astype(float) > 100]
#food_sugars_outlier




,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,packaging,packaging_tags,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,no_nutriments,additives_n,additives,additives_tags,additives_en,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,ingredients_that_may_be_from_palm_oil_tags,nutrition_grade_uk,nutrition_grade_fr,pnns_groups_1,pnns_groups_2,states,states_tags,states_en,main_category,main_category_en,image_url,image_small_url,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,-butyric-acid_100g,-caproic-acid_100g,-caprylic-acid_100g,-capric-acid_100g,-lauric-acid_100g,-myristic-acid_100g,-palmitic-acid_100g,-stearic-acid_100g,-arachidic-acid_100g,-behenic-acid_100g,-lignoceric-acid_100g,-cerotic-acid_100g,-montanic-acid_100g,-melissic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,-alpha-linolenic-acid_100g,-eicosapentaenoic-acid_100g,-docosahexaenoic-acid_100g,omega-6-fat_100g,-linoleic-acid_100g,-arachidonic-acid_100g,-gamma-linolenic-acid_100g,-dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,-oleic-acid_100g,-elaidic-acid_100g,-gondoic-acid_100g,-mead-acid_100g,-erucic-acid_100g,-nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,-sucrose_100g,-glucose_100g,-fructose_100g,-lactose_100g,-maltose_100g,-maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g
94265,8005305900255,http://world-en.openfoodfacts.org/product/8005...,mathias,1430147873,2015-04-27T15:17:53Z,1433016494,2015-05-30T20:08:14Z,Ekstra Jomfru Olivenolie,NaN,1L,NaN,NaN,Santagata,santagata,NaN,NaN,NaN,Europe,europe,Italy,italy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kvickly,Denmark,en:denmark,Denmark,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,e,unknown,unknown,"en:to-be-completed, en:nutrition-facts-complet...","en:to-be-completed,en:nutrition-facts-complete...","To be completed,Nutrition facts completed,Ingr...",NaN,NaN,http://en.openfoodfacts.org/images/products/80...,http://en.openfoodfacts.org/images/products/80...,3737,NaN,101,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20,20


In [6]:
food_countries['allergens'] = food_countries['allergens'].str.split(',').apply(len)
food_countries['additives_n'] = food_countries['additives_n'].astype(int)
food_countries
#countries_final = np.ndarray.sort(countries_temp)
#countries_final

,-alpha-linolenic-acid_100g,-arachidic-acid_100g,-arachidonic-acid_100g,-behenic-acid_100g,-butyric-acid_100g,-capric-acid_100g,-caproic-acid_100g,-caprylic-acid_100g,-cerotic-acid_100g,-dihomo-gamma-linolenic-acid_100g,-docosahexaenoic-acid_100g,-eicosapentaenoic-acid_100g,-elaidic-acid_100g,-erucic-acid_100g,-fructose_100g,-gamma-linolenic-acid_100g,-glucose_100g,-gondoic-acid_100g,-lactose_100g,-lauric-acid_100g,-lignoceric-acid_100g,-linoleic-acid_100g,-maltodextrins_100g,-maltose_100g,-mead-acid_100g,-melissic-acid_100g,-montanic-acid_100g,-myristic-acid_100g,-nervonic-acid_100g,-oleic-acid_100g,-palmitic-acid_100g,-stearic-acid_100g,-sucrose_100g,additives,additives_en,additives_n,additives_tags,alcohol_100g,allergens,allergens_en,beta-carotene_100g,bicarbonate_100g,biotin_100g,brands,brands_tags,caffeine_100g,calcium_100g,carbohydrates_100g,carbon-footprint_100g,casein_100g,categories,categories_en,categories_tags,chloride_100g,chlorophyl_100g,cholesterol_100g,chromium_100g,cities,cities_tags,cocoa_100g,code,collagen-meat-protein-ratio_100g,copper_100g,countries,countries_en,countries_tags,created_datetime,created_t,creator,emb_codes,emb_codes_tags,energy-from-fat_100g,energy_100g,fat_100g,fiber_100g,first_packaging_code_geo,fluoride_100g,fruits-vegetables-nuts_100g,generic_name,image_small_url,image_url,ingredients_from_palm_oil,ingredients_from_palm_oil_n,ingredients_from_palm_oil_tags,ingredients_text,ingredients_that_may_be_from_palm_oil,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil_tags,iodine_100g,iron_100g,labels,labels_en,labels_tags,last_modified_datetime,last_modified_t,magnesium_100g,main_category,main_category_en,manganese_100g,manufacturing_places,manufacturing_places_tags,molybdenum_100g,monounsaturated-fat_100g,no_nutriments,nucleotides_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,nutrition_grade_fr,nutrition_grade_uk,omega-3-fat_100g,omega-6-fat_100g,omega-9-fat_100g,origins,origins_tags,packaging,packaging_tags,pantothenic-acid_100g,ph_100g,phosphorus_100g,pnns_groups_1,pnns_groups_2,polyols_100g,polyunsaturated-fat_100g,potassium_100g,product_name,proteins_100g,purchase_places,quantity,salt_100g,saturated-fat_100g,selenium_100g,serum-proteins_100g,serving_size,silica_100g,sodium_100g,starch_100g,states,states_en,states_tags,stores,sugars_100g,taurine_100g,traces,traces_en,traces_tags,trans-fat_100g,url,vitamin-a_100g,vitamin-b12_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-b6_100g,vitamin-b9_100g,vitamin-c_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-pp_100g,zinc_100g
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ sucre -> fr:sucre ] [ farine-de-ble -> fr...,"E1400 - Dextrin,E322 - Lecithins,E503 - Ammoni...",3,"en:e1400,en:e322,en:e503",NaN,4,NaN,NaN,NaN,NaN,St Michel,st-michel,NaN,NaN,NaN,NaN,NaN,sablés,"Sugary snacks,Biscuits and cakes,Biscuits,fr:S...","en:sugary-snacks,en:biscuits-and-cakes,en:bisc...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0000007730009,NaN,NaN,France,France,en:france,2016-05-09T15:07:12Z,1462806432,openfoodfacts-contributors,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Biscuits sablés fourrage au cacao,http://en.openfoodfacts.org/images/products/00...,http://en.openfoodfacts.org/images/products/00...,NaN,0,NaN,"Sucre, farine de _Blé_, graisse et huiles végé...",NaN,1,huile-vegetale,NaN,NaN,NaN,NaN,NaN,2016-05-09T16:07:17Z,1462810037,NaN,en:sugary-snacks,Sugary snacks,NaN,France,france,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Carton,carton,NaN,NaN,NaN,Sugary snacks,Biscuits and cakes,NaN,NaN,NaN,Biscuits sablés fourrage au cacao,NaN,NaN,1 kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"en:to-be-completed, en:nutrition-facts-to-be-c...","To be completed,Nutrition facts to be complete...","en:to-be-completed,en:nutrition-facts-to-be-co...",NaN,NaN,NaN,"Fruits à coque,Soja,Graines de sésame","Nuts,Sesame seeds,Soybeans","en:nuts,en:sesame-seeds,en:soybeans",NaN,ht

In [7]:
#food_countries.groupby('countries_en', as_index=False)['allergens'].mean()
final_data = food_countries.groupby('countries_en', as_index=False)['allergens','additives_n'].mean()

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows',100)

plot_data = final_data.sort(['allergens', 'additives_n'], ascending=[0, 0])

plot_data

/home/vivek-sethia/.local/lib/python3.5/site-packages/ipykernel/__main__.py:7: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,countries_en,allergens,additives_n
37,Morocco,7.000000,4.000000
53,Slovenia,7.000000,2.500000
19,French Polynesia,6.333333,4.666667
49,Saudi Arabia,6.000000,10.000000
20,Gabon,6.000000,3.000000
51,Serbia,6.000000,3.000000
0,Algeria,6.000000,1.000000
59,Tunisia,5.333333,0.666667
26,Indonesia,5.000000,12.000000
36,Montenegro,5.000000,3.000000


In [22]:
import plotly.plotly as py
import plotly.graph_objs as go
py.sign_in('viveksethia', 'y0nvnlhe12')


trace1 = go.Bar(
    x=plot_data['countries_en'][:50],
    y=plot_data['allergens'],
    name='Allergens'
)
trace2 = go.Bar(
    x=plot_data['countries_en'][:50],
    y=plot_data['additives_n'],
    name='Additives'
)

data = [trace1, trace2]
layout = go.Layout(
    #xaxis=dict(tickangle=-180),
    barmode='group',
    height=700,
    margin=go.Margin(
        l=50,
        r=50,
        b=150,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='grouped-bar-1', bbox_inches='tight')

In [ ]:
food